## Model API

Iris 데이터를 입력받아 예측값을 반환하는 API 를 작성합니다.작성한 API 에 데이터를 전달하여 제대로 작동하는지 확인합니다.

## **스펙 명세서**

1. `03. Model Registry`  파트에서 모델을 학습한 후 저장한 MLflow 서버의 Model Registry 에서 모델을 로컬로 다운로드받는 스크립트 `download_model.py` 를 작성하고 실행합니다.
2. `POST /predict` 를 수행하면 학습한 모델의 inference 결과를 반환하는 API 의 명세서를 작성합니다.
3. `schemas.py` 에서 Pydantic 을 사용해 input schema 와 output schema 의 클래스를 작성합니다.
    - Input schema: `Class PredictIn(BaseModel)` 을 이용
        - Column 이름: 01. Database 파트에서 작성한 이름과 동일
    - Output schema: `Class PredictOut(BaseModel)` 을 이용
        - Column 이름: `iris_class`작성한 명세서를 FastAPI 를 이용해 `app.py` 에 구현합니다.작성한 API 에 데이터를 전달하여 잘 작동하는지 확인합니다.

해당 파트의 전체 코드는 [mlops-for-mle/part6/](https://github.com/mlops-for-mle/mlops-for-mle/tree/main/part6) 에서 확인할 수 있습니다.

## 환경 설정

- 이번 파트에서 사용할 패키지들을 설치합니다

In [1]:
# terminal-command
!pip install boto3==1.26.8 mlflow==1.30.0 "fastapi[all]" pandas scikit-learn


# 1. 모델 다운로드

### 1.1 Base Setting
먼저 필요한 패키지들을 import 합니다.

In [2]:
import os
from argparse import ArgumentParser

import mlflow

### 1.2 Environment Variables

다음으로는 Model Registry 에 저장되어 있는 모델을 다운로드하기 위해 MLflow 서버와 MinIO 서버에 접속하기 위한 정보를 환경 변수로 설정합니다.

In [3]:
# Set environments
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5001"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "miniostorage"

### 1.3 모델 다운로드 함수 작성

In [4]:
def download_model(args):# Download model artifacts
    mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{args.run_id}/{args.model_name}", dst_path=".")


### 1.4 모델 다운로드

In [6]:
# if __name__ == "__main__":
#     parser = ArgumentParser()
#     parser.add_argument("--model-name", dest="model_name", type=str, default="sk_model")
#     parser.add_argument("--run-id", dest="run_id", type=str)
#     args = parser.parse_args()
# 
#     download_model(args)


### 1.5 download_model.py

In [8]:
%%writefile download_model.py

import os
from argparse import ArgumentParser

import mlflow

# Set environments
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5001"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "miniostorage"

def download_model(args):# Download model artifacts
    mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{args.run_id}/{args.model_name}", dst_path=".")


if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--model-name", dest="model_name", type=str, default="sk_model")
    parser.add_argument("--run-id", dest="run_id", type=str)
    args = parser.parse_args()

    download_model(args)


Overwriting download_model.py


### 1.6 스크립트 실행

먼저 다운로드받고자 하는 모델의 MLFlow 서버에서 저장된 **`run_id`** 와 **`model_name`** 을 알아야 합니다.

[http://localhost:5001](http://localhost:5001/) 에 접속하여 모델이 저장된 **`experiments`** 와 **`run`** 을 선택하여 클릭합니다. [그림 6-2]의 빨간색 상자 부분에서 **`run_id`** 와 **`model_name`** 을 각각 확인할 수 있습니다.

    # terminal-command
    python download_model.py --model-name sk_model --run-id <run-id>

In [ ]:
run_id = '3bf7aa18a6c8473d8a287f414698353e'

In [ ]:
python download_model.py --model-name sk_model --run-id 3bf7aa18a6c8473d8a287f414698353e

# 2. Model API 명세서 작성

POST /predict 를 수행했을 때 학습한 모델의 inference 결과를 반환해주는 API 의 명세서를 작성합니다. Request Body 로 iris 데이터를 전달해주면 Response Body 를 통해 예측된 값을 전달받게 됩니다.


# 3. Pydantic Model 로 스키마의 클래스 작성

In [9]:
%%writefile schemas.py
#schemas.py
from pydantic import BaseModel

class PredictIn(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

class PredictOut(BaseModel):
    iris_class: int

Writing schemas.py


# 4. Predict API 구현

In [11]:
%%writefile app.py
# app.py
import mlflow
import pandas as pd
from fastapi import FastAPI
from schemas import PredictIn, PredictOut


def get_model():
    model = mlflow.sklearn.load_model(model_uri="./sk_model")
    return model


MODEL = get_model()

# Create a FastAPI instance
app = FastAPI()


@app.post("/predict", response_model=PredictOut)
def predict(data: PredictIn) -> PredictOut:
    df = pd.DataFrame([data.dict()])
    pred = MODEL.predict(df).item()
    return PredictOut(iris_class=pred)

Writing app.py


# 5. API 작동 확인
    
    # terminal-command
    uvicorn app:app --reload    

# Model API on Docker Compose

# 1. Dockerfile 작성

In [12]:
%%writefile Dockerfile
FROM --platform=linux/arm64 python:3.9-slim

WORKDIR /usr/app

RUN pip install -U pip &&\

    pip install mlflow==1.30.0 pandas scikit-learn "fastapi[all]"

COPY schemas.py schemas.py
COPY app.py app.py
COPY sk_model/ sk_model/

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--reload"]

Writing Dockerfile


#  2. Docker Compose

In [13]:
%%writefile docker-compose.yaml
version: "3"

services:
  api-with-model:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: api-with-model
    ports:
      - 8000:8000
    healthcheck:
      test:
        - CMD
        - curl -X POST http://localhost:8000/predict
        - -H
        - "Content-Type: application/json"
        - -d
        - '{"sepal_length": 6.7, "sepal_width": 3.3, "petal_length": 5.7, "petal_width": 2.1}'
      interval: 10s
      timeout: 5s
      retries: 5

networks:
  default:
    name: mlops-network
    external: true

Writing docker-compose.yaml
